# Lecture 3: Constraint/Column Generation and the Cutting Stock Problem

In this Jupyter Notebook, we will be exploring the cutting stock problem using Python and Gurobi. We will begin by solving the problem by simply enumerating patterns. We will then see three different approaches: (exact) constraint generation, heuristic constraint generation, and constraint randomization/sampling. 

## The cutting stock problem with two patterns

In our initial formulation of this problem, we are told that we have 3 types of demand: 3 inch rolls; 5 inch rolls; 7 inch rolls. We need to meet demand for these 3 types by cutting large rolls of 20 inches according to _patterns_.

We are told that we will only consider two types of patterns:
- Pattern 1: [3, 2, 0] (3 x 3 inch rolls; 2 x 5 inch rolls; 0 x 7 inch rolls)
- Pattern 2: [2, 1, 1] (2 x 3 inch rolls; 1 x 5 inch rolls; 1 x 7 inch rolls)

We are told that we need:
- 22 rolls of type 1 (3 inches)
- 10 rolls of type 2 (5 inches)
- 5 rolls of type 3 (7 inches)

We can formulate the problem as the following linear program. Let $x_1$ be the number of large rolls we cut according to pattern 1, and similarly let $x_2$ be the number of large rolls cut according to pattern 2. With the goal of minimizing the total number of large rolls that we use, we can write the problem down as the following linear program:

$$
\begin{align}
& \text{minimize} & & x_1 + x_2 \\
& \text{subject to} & & 3 x_1 + 2 x_1 \geq 22, \\
& & & 2 x_1 + x_2 \geq 10, \\
& & & x_2 \geq 5, \\
& & & x_1, x_2 \geq 0.
\end{align}
$$

Note that the above formulation allows for the number of patterns to be cut to be fractional. As we have seen in some of our LP formulations earlier, we will not worry about this. 

Let's now formulate this in Python/Gurobi.


In [1]:
from gurobipy import *
import numpy as np

nDemands = 3
d = np.array([22, 10, 5])

nPatterns = 2
patterns = [ [3,2,0], [2,1,1]]

m = Model()
x = m.addVars(nPatterns)

for i in range(nDemands):
    m.addConstr(sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
    
m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

m.update()

m.optimize()

x_vals = [ x[j].x for j in range(nPatterns)]

print("Optimal solution: ", x_vals)

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x88576948
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    9.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.000000000e+00
Optimal solution:  [4.0, 5.0]


So our optimal solution is to cut 4 rolls according to pattern 1, and 5 rolls according to pattern 2. (Note that although we were lucky here and the optimal values came out as integers, this will not be the case in general. As an example, if you change `d[0]` to 20, you will get a fractional number of rolls for the first pattern.) 

## The cutting stock problem with a large number of patterns (exhaustive enumeration)

In the previous example, the two patterns were chosen arbitrarily. In reality, we are free to cut the large rolls according to any pattern we like.

So now, suppose that we enumerate all of the possible patterns. Mathematically, this would mean that we have $n$ patterns, where each pattern $j \in \{1,\dots,n\}$ is a vector $\mathbf{a}_j = (a_{1,j}, a_{2,j}, \dots, a_{m,j})$ that tells us how many rolls of each demand type will be cut from the large rolls. 

The LP can then be written as 
$$
\begin{alignat}{2}
& \text{minimize} & \quad & \sum_{j=1}^n x_j \\
& \text{subject to} & & \sum_{j=1}^n a_{1,j} x_j \geq d_1, \\
& & & \sum_{j=1}^n a_{2,j} x_j \geq d_2, \\
& & & \vdots \\
& & & \sum_{j=1}^n a_{m,j} x_j \geq d_m, \\
& & & x_1, \dots, x_n \geq 0.
\end{alignat}
$$

Let's now see how we can enumerate all of the patterns in Python and then formulate the LP using Gurobi. 

In [ ]:
# Define the widths of the large roll and the small rolls:
W = 20 # width of a large roll 
w = [3, 5, 7] # widths of the three different small rolls

# In Python, // will perform integer / "floor" division (do an ordinary division, 
# and then take the floor() of the result)
print("W // w[0]: ", W // w[0])

a_ranges = [ list(range( W // w[i] + 1 ) ) for i in range(nDemands)]
print(a_ranges)

# To obtain the patterns, let's use a simple list comprehension with a condition:
patterns = [ (i,j,k) for i in a_ranges[0] for j in a_ranges[1] for k in a_ranges[2] if w[0]*i+w[1]*j+w[2]*k<=W]

# Count the number of patterns:
nPatterns = len(patterns)
print("nPatterns = ", nPatterns)

# Notice that this is much smaller than the theoretical upper bound of 105!

# Let's proceed with formulating the problem:

m = Model()
x = m.addVars(nPatterns)

for i in range(nDemands):
    m.addConstr( sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
    
m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

m.update()

m.optimize()

x_vals = [ x[j].x for j in range(nPatterns)]
print()
print("Optimal solution: ", x_vals)
print()
nonzero_pattern_indices = [j for j in range(nPatterns) if x_vals[j] > 0]

for j in nonzero_pattern_indices:
    print("Pattern ", j, ": ", patterns[j], " -- cut ", x_vals[j], " rolls")

## Dual of the cutting stock problem

Of course, we can also solve this same problem using the dual:

$$
\begin{alignat}{2}
& \text{maximize} & & \sum_{i=1}^m d_i p_i \\
& \text{subject to} & \quad & \sum_{i=1}^m a_{i,1} p_i \leq 1, \\
& & & \sum_{i=1}^m a_{i,2} p_i \leq 1, \\
& & & \vdots \\
& & & \sum_{i=1}^m a_{i,n} p_i \leq 1, \\
& & & p_1,\dots, p_m \geq 0. 
\end{alignat}
$$

Let's formulate this problem in Gurobi below:

In [ ]:
m_dual = Model()
p = m_dual.addVars(nDemands)

for j in range(nPatterns):
    m_dual.addConstr( sum( patterns[j][i] * p[i] for i in range(nDemands) ) <= 1)
    
m_dual.setObjective( sum( d[i] * p[i]  for i in range(nDemands)), GRB.MAXIMIZE)

m_dual.update()

m_dual.optimize()

## Solving the cutting stock problem using constraint generation

In the previous example, we only had a small number of demand types and the number of possible patterns was small enough that we could enumerate it completely, and solve the problem. In general, this will not be feasible when we have a large number of demand types, and there is an enormous number of patterns to consider.

In what follows below, we are going to implement a constraint generation approach for solving this problem. The idea is that we are going to solve the dual, check for violated constraints, and then add them as needed.

To begin, let us set the width and demand parameters to the ones of our larger example from the slides:

In [ ]:
W = 200
w = [3,5,7,10,17,22,30,50]
nDemands = 8
# d = np.ones(nDemands) * 2000 
d = [1200,1000,1000,400,500,400,600,200]

Now, we are going to formulate the separation problem. Recall that the separation problem is the problem that we solve to determine if there is a violated constraint. 

In [ ]:
# "sub" indicates that this is the subproblem in our CG method.
m_sub = Model() 

a = m_sub.addVars(nDemands, vtype = GRB.INTEGER)
p_val = np.ones(nDemands)

m_sub.addConstr( sum(w[i] * a[i] for i in range(nDemands)) <= W)
m_sub.setObjective( sum(p_val[i] * a[i] for i in range(nDemands)), GRB.MAXIMIZE)

m_sub.update()

# Test it out:
m_sub.optimize()

Great! The subproblem is working. Let's actually create a function now, to help us solve it repeatedly.

In [ ]:
def separation(p_val):
    m_sub.setObjective( sum(p_val[i] * a[i] for i in range(nDemands)), GRB.MAXIMIZE)
    m_sub.update()
    m_sub.optimize()
    
    a_val = [a[i].x for i in range(nDemands)]
    sub_obj = m_sub.objval
    
    return sub_obj, a_val

# Test it out (set OutputFlag to zero, to not flood notebook with output:)
m_sub.Params.OutputFlag = 0

sub_obj, a_val = separation( [2, 3, 5, 0, 0, 0, 0, 0] )
print(sub_obj, a_val)

sub_obj, a_val = separation( [2, 2, 2, 1, 5, 1, 1, 1] )
print(sub_obj, a_val)

We can now create our constraint generation algorithm.

In [ ]:
# Reformulate the dual problem. 
m_dual = Model()

m_dual.Params.OutputFlag = 0

p = m_dual.addVars(nDemands)

m_dual.setObjective(sum(d[i] * p[i] for i in range(nDemands)), GRB.MAXIMIZE)

# Let us initialize the set of patterns with just those that consist only of a single demand type.
# We can enumerate that set as follows:

patterns = [ [1 for i in range(nDemands)] ]
print(patterns)

nPatterns = len(patterns)

for j in range(nPatterns):
    m_dual.addConstr( sum( patterns[j][i] * p[i] for i in range(nDemands)) <= 1)
    
m_dual.update()
m_dual.optimize()



dual_obj = m_dual.objval

CG_iter = 0

print("Iteration ", CG_iter, ": ", dual_obj)


p_val = [p[i].x for i in range(nDemands)]

print(p_val)



sub_objval, a_val = separation(p_val)

tol_eps = 1e-5

isOptimal = sub_objval <= 1 + tol_eps # Why do we do this?

while (not isOptimal):
    # Add a_val to patterns, and add the constraint to the dual.
    patterns.append(a_val)
    m_dual.addConstr( sum( a_val[i] * p[i] for i in range(nDemands)) <= 1)
    
    # Update the problem and solve it again.
    m_dual.update()
    m_dual.optimize()
    dual_obj = m_dual.objval
    p_val = [p[i].x for i in range(nDemands)]
    
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, a_val = separation(p_val)
    isOptimal = sub_objval <= 1 + tol_eps
    print("sub_objval = ", sub_objval)
    print("constraint violation = ", 1 - sub_objval),
    print("a_val = ", a_val)
    print("isOptimal = ", isOptimal)
    

To figure out the solution to the original problem, let's formulate the primal and solve again (the cell below is just reusing some code from earlier):

In [ ]:
nPatterns = len(patterns)

m = Model()
x = m.addVars(nPatterns)

for i in range(nDemands):
    m.addConstr( sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
    
m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

m.update()

m.optimize()

x_vals = [ x[j].x for j in range(nPatterns)]

print("Optimal solution: ", x_vals)

nonzero_pattern_indices = [j for j in range(nPatterns) if x_vals[j] > 0]

for j in nonzero_pattern_indices:
    print("Pattern ", j, ": ", patterns[j], " -- cut ", x_vals[j], " rolls")
    

## Heuristic constraint generation
Instead of solving the separation problem as an integer program, we can also consider solving it using a heuristic. Consider the following heuristic:
1. Given the $p_i$ values, calculate a index $r_i = p_i / w_i$. 
2. Sort the demand types from largest to smallest value of the index $r_i$, so that $r_{(1)} \geq r_{(2)} \geq \dots \geq r_{(m)}$. 
3. For $i = 1,\dots,m$, set 
$a_{(i)} =  \left \lfloor \frac{W - \sum_{i'=1}^{i-1} a_{(i')} w_{(i')}}{w_{(i)}} \right\rfloor $.

This type of heuristic is known as a "bang-for-buck" heuristic. The index $r_i$ tells us what is the "bang-for-buck" of each demand type (how much it adds to the separation problem objective per inch it uses of the large roll). The heuristic then builds the pattern in order of this index. 

In [ ]:
def separation_heuristic(p_val):
    r = [ p_val[i] / w[i] for i in range(nDemands) ]
    order = np.argsort( r )
    order = np.flip(order)
    
    a_val = np.zeros(nDemands)
    remaining_width = W
    for i in order:
        a_val[i] = remaining_width // w[i]
        remaining_width -= a_val[i] * w[i]
        
    sub_objval = sum( [ a_val[i] * p_val[i] for i in range(nDemands) ])
    
    return sub_objval, a_val


p_val = [0.05, 0.01, 0.03, 0.02]
print( np.argsort(p_val))


m_dual = Model()

m_dual.Params.OutputFlag = 0

p = m_dual.addVars(nDemands)

m_dual.setObjective(sum(d[i] * p[i] for i in range(nDemands)), GRB.MAXIMIZE)

# Let us initialize the set of patterns with just those that consist only of a single demand type.
# We can enumerate that set as follows:

patterns = [ [1 for i in range(nDemands)] ]
print(patterns)

nPatterns = len(patterns)

for j in range(nPatterns):
    m_dual.addConstr( sum( patterns[j][i] * p[i] for i in range(nDemands)) <= 1)
    
m_dual.update()
m_dual.optimize()



dual_obj = m_dual.objval

CG_iter = 0

print("Iteration ", CG_iter, ": ", dual_obj)


p_val = [p[i].x for i in range(nDemands)]

sub_objval, a_val = separation_heuristic(p_val)

tol_eps = 1e-5

isOptimal = sub_objval <= 1 + tol_eps

while (not isOptimal):
    # Add a_val to patterns, and add the constraint to the dual.
    patterns.append(a_val)
    m_dual.addConstr( sum( a_val[i] * p[i] for i in range(nDemands)) <= 1)
    
    # Update the problem and solve it again.
    m_dual.update()
    m_dual.optimize()
    dual_obj = m_dual.objval
    p_val = [p[i].x for i in range(nDemands)]
    
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, a_val = separation_heuristic(p_val)
    isOptimal = sub_objval <= 1 + tol_eps
    print("sub_objval = ", sub_objval)
    print("constraint violation = ", 1 - sub_objval),
    print("a_val = ", a_val)
    print("isOptimal = ", isOptimal)
    


In [ ]:
sub_objval, a_val = separation(p_val)
print(sub_objval)

You can see that this heuristic-based constraint generation method terminates prematurely; in the final iteration, the objective value is 326.41, but we know that the optimal is actually 324.5. 

For many problems, this type of heuristic constraint generation approach will work well -- it will get us the optimal or a near optimal solution, and it can be potentially much faster than when we solve the separation problem as an IP. We can also consider a hybrid scheme, where we first run the heuristic: if the heuristic finds a violated constraint, we add it and solve again; otherwise, if the heuristic indicates that no constraints are violated, we trigger a solve of the integer program to confirm that this is truly the case. 

## Randomization approach

This last example is to illustrate a very different approach to solving large scale LPs, which is based on randomly sampling columns. This is actually a topic of some of my recent research:

> Chen, Y.-C. and Misic, V. V. (2020) Column-Randomized Linear Programs: Performance Guarantees and Applications. _Working paper; available online at https://arxiv.org/abs/2007.10461._

The idea here is quite simple: rather than adding constraints one at a time, let's _randomly sample_ a collection of constraints, and solve the problem. 

The function below will randomly sample a pattern. The procedure starts with an empty pattern, and then randomly selects one of the demand types to add to the pattern. It keeps repeating this until it runs out of space on the large roll to add any more demand types. 

In [ ]:
from random import sample

min_w = min(w)

def random_a():
    a_val = np.zeros(nDemands)
    remaining_width = W
    valid_widths = [i for i in range(nDemands) if w[i] <= remaining_width]
    while (len(valid_widths) > 0):
        i_random = np.random.choice(valid_widths, 1)
        i_random = i_random[0]
        a_val[i_random] += 1
        remaining_width -= w[i_random]
        valid_widths = [i for i in range(nDemands) if w[i] <= remaining_width]
        
    return a_val

# Test it out:
print(random_a())
print(random_a())
print(random_a())

In [ ]:
# Let's now implement the randomization procedure:

np.random.seed(100) #100

# Sample 100 random patterns:
nPatterns = 100
patterns = []
for j in range(nPatterns):
    patterns.append( random_a() )
    
# Formulate the dual as per usual:  
m_dual = Model()

# m_dual.Params.OutputFlag = 0

p = m_dual.addVars(nDemands)

m_dual.setObjective(sum(d[i] * p[i] for i in range(nDemands)), GRB.MAXIMIZE)

for j in range(nPatterns):
    m_dual.addConstr( sum( patterns[j][i] * p[i] for i in range(nDemands)) <= 1)
    
m_dual.update()
m_dual.optimize()

Somewhat surprisingly, this randomized method gets us a solution that is quite close to optimal (325.16, compared to 324.5). Let us see what the actual patterns/allocations look like:

In [ ]:
# Solve the primal problem to see which patterns are involved:
m = Model()
x = m.addVars(nPatterns)

m.Params.OutputFlag = 0

for i in range(nDemands):
    m.addConstr( sum( patterns[j][i] * x[j] for j in range(nPatterns) ) >= d[i])
m.setObjective( sum(x[j] for j in range(nPatterns)), GRB.MINIMIZE)

m.update()
m.optimize()

x_vals = [ x[j].x for j in range(nPatterns)]
nonzero_pattern_indices = [j for j in range(nPatterns) if x_vals[j] > 0]
print("Objective value: ", m.objval)
for j in nonzero_pattern_indices:
    print("Pattern ", j, ": ", patterns[j], " -- cut ", x_vals[j], " rolls")

Notice that these patterns look very different from the ones in our exact solution. (In the exact solution, the patterns are much sparser.) 

This provides us some insight into why this randomization method works (unreasonably) well -- there are lots of nearly optimal solutions to the true problem that use different patterns. Although there may just be one singular optimal solution, by randomly sampling patterns, we are likely to sample some of the patterns that participate in the nearly optimal solutions, and thus be able to form a nearly optimal solution. 